In [1]:
# classifier: cats vs. dogs
# dataset: https://www.kaggle.com/c/dogs-vs-cats

# ---------------------
# import required packages
# ---------------------
import cv2
import csv
import numpy as np
import os
import matplotlib.pyplot as plt
import torch
from random import shuffle
from tqdm import tqdm
import torchvision
from sklearn.linear_model import LogisticRegression

In [2]:
# ---------------------
# define constants
# ---------------------
'''from keras.datasets import mnist
trainset= (X_train, y_train)=  mnist.load_data()
testset =(X_test, y_test) =  mnist.load_data()'''

trainset = ('/home/rabina7/Downloads/MNIST/trainset.csv')
testset = ('/home/rabina7/Downloads/MNIST/testset.csv')
with open(trainset, 'r') as csvfile: 
    # creating a csv reader object 
    csvreader = csv.reader(csvfile) 
with open(testset, 'r') as csvfile: 
    # creating a csv reader object 
    csvreader = csv.reader(csvfile) 
    
#filters out every digits except 0 and 1
train_filter = np.where((trainset == 0) | (trainset == 1))
test_filter = np.where((testset == 0) | (testset == 1))
    
IMG_SIZE = 28 # resize all images to the same size
LR = 1e-3 # learning rate: 0.001

# model name, for model loading and saving
MODEL_NAME = 'MNIST-{}-{}.model'.format(LR, '6conv-basic')

In [3]:
# output of our model is an array consists 2 elements: [catniss, dogniss]
# each element represents the probability of being each animal.
# e.g., [1, 0] means this image is 100% a cat image and 0% a dog image.
# So, the final label of the image is cat.

categories = ['0', '1']

def img_label(img):
    # get the true label of the image
    # images in training set are named in form of <animal>.xx.jpg, xx are numbers.
    print("img path: "+img)
    parts = img.split('.')
    print("length of parts: "+str(len(parts)))
    print("parts: "+str(parts))
    
    text_label = img.split('.')[-1]
    
    if text_label == categories[0]:
        return [0, 1]
    elif text_label == categories[1]:
        return [1, 0]
    else:
        raise ValueError('The image does not belong to any category! ' + text_label)

In [4]:
# ---------------------
# process training data
# ---------------------
def process_train_data():
    training_data = []
    for img in tqdm(os.path.join(trainset)):
        # full path of the image
        img_full_path = os.path.join(trainset, img)
        # true label of the image
        true_label = img_label(img)
        # load the image
        img = cv2.imread(img_full_path, cv2.IMREAD_GRAYSCALE)
        # resize the image
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        # add new training data, in form of
        # (img, label)
        training_data.append([np.array(img), np.array(true_label)])
        #training_data.append([np.array(img), img_idx])
    # shuffle the dataset
    shuffle(training_data)
    # save the dataset so that next time we can load it directly
    np.save('trainset-{}.npy'.format(IMG_SIZE), training_data)
    
    return training_data       

In [5]:
# ---------------------
# process testing data
# ---------------------
def process_test_data():
    testing_data = []
    for img in tqdm(os.listdir(testset)):
        # full path of the image
        img_full_path = os.path.join(testset, img)
        #img_idx = img.split('.')[0]
        true_label = img_label(img)
         # load the image
        img = cv2.imread(img_full_path, cv2.IMREAD_GRAYSCALE)
        # resize the image
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        # add new testing data, in form of
        # (img, img_idx)
        testing_data.append([np.array(img), np.array(true_label)])
        #calculating the testing
        #simple loop that goes through all the 
        #API to predict the label and compare to the true label
        #counter
    shuffle(testing_data)
    # save the dataset so that next time we can load it directly
    np.save('testset-{}.npy'.format(IMG_SIZE), testing_data)
    return testing_data

In [6]:
# ---------------------
# load training data
# ---------------------
def load_train_data(load_exist):
    trainset = None
    if load_exist and os.path.exists('trainset-{}.npy'.format(IMG_SIZE)):
        # if want to load an existing training set
        trainset = np.load('trainset-{}.npy'.format(IMG_SIZE), allow_pickle = True)
    else:
        trainset = process_train_data()
    return trainset

In [7]:
# ---------------------
# construct the cnn model for this project
# ---------------------
import tflearn
import tensorflow as tf

from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected, flatten
from tflearn.layers.estimator import regression

def conv_net():
    convnet = input_data(shape = [None, IMG_SIZE, IMG_SIZE, 1],
                        name = 'input')
    # conv_2d(incoming, nb_filter, filter_size, ..., activation)
    convnet = conv_2d(convnet, 32, 5, activation = 'relu')
    # max_pool_2d(incoming, kernel_size, ...)
    convnet = max_pool_2d(convnet, 5)
    
    convnet = conv_2d(convnet, 64, 5, activation = 'relu')
    convnet = max_pool_2d(convnet, 5)
   
    #flatten the layer
    flatten = tflearn.flatten(convnet)
    # fully_connected(incoming, n_units, activation, ...)
    convnet = fully_connected(convnet, 1024, activation = 'relu')
    # dropout(incoming, keep_prob is drop_prob + keep+prob)
    convnet = dropout(convnet, 0.8)
    
    #standard recommendation for the Net arch    
    logits = fully_connected(convnet, 2, activation = 'softmax')
    #convnet = dropout(convnet, 0.8)
    
    #logits
    #logits = tf.layers.dense(inputs=convnet, units=10)
    
    #regression(incoming, optimizer, learning_rate, loss, name, ...)
    convnet = regression(logits, optimizer = 'adam', learning_rate = LR,
                             loss = 'categorical_crossentropy', name = 'targets')

    return convnet

Instructions for updating:
Colocations handled automatically by placer.


In [8]:
# ---------------------
# define the model
# ---------------------
convnet = conv_net()
model = tflearn.DNN(convnet, tensorboard_dir = 'log')


Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use tf.cast instead.


In [9]:
# ---------------------
# train the model
# ---------------------

# load model if exists
if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print ('loaded existing model', MODEL_NAME)
else:
    # Loading the training data-set after pre-processing
    # and splitting the testing data and validation data
    
    train_data = load_train_data(False)
    print(np.array(train_data).shape)
    divide = (len(train_data)/5)
    for i in range(5):
        start_id = (divide * i)
        print("startid: " + str(start_id))
        end_id = (divide * (i + 1) -1)
        print("endid: " + str(end_id))
        validation = train_data[start_id: end_id]
        if (i == 0):
            train = train_data[end_id +1 : (len(train_data)-1)]
            #print("Train0:" + str(train))
        if (i == 1 and i == 2 and i == 3):
            train = train_data[0: (start_id)-1].merge[end_id +1 : (len(train_data)-1)]
        if(i == 4):
            train = train = train_data[0 : start_id -1]
        # Setting up the features and lables
        # x-Features & y-Labels
        train_x = np.array([i[0] for i in train]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
        train_y = [i[1] for i in train]
        validation_x = np.array([i[0] for i in validation]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
        validation_y = [i[1] for i in validation]
        # Training the CNN model with the data-sets
        model.fit({'input': train_x}, {'targets': train_y}, n_epoch = 3,
                            validation_set = ({'input': validation_x}, {'targets': validation_y}),
                            snapshot_step = 500, show_metric = True, run_id = MODEL_NAME)

    # save the trained model
    #model.save(MODEL_NAME)
    #enough training/testing sample of each class
    #JSMA
    #debug on a different MNISTdataset/ standard arch

  0%|          | 0/42 [00:00<?, ?it/s]


img path: /
length of parts: 1
parts: ['/']


ValueError: The image does not belong to any category! /

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
'''refined_x=train_x.flatten().reshape(7505, 2500)
train_y_labels=[]
for i in range(len(train_y)):
    train_y_labels.append(train_y[i][0])
    train_y_labels=np.array(train_y_labels)
    logisticRegr = LogisticRegression()
    logisticRegr.fit(refined_x,train_y_labels)'''

In [ ]:
'''prediction=logisticRegr.predict(refined_x)
count=0
total_train = len(prediction)
for i in range(len(prediction)):
    if (prediction[i]==train_y_labels[i]):
        print("prediction{}:, Train{}: ".format(prediction, train_y_labels))
        count=count+1 
accuracy = (1.0 * count / total_train)             
print("The training accuracy is {}.".format(accuracy))'''

In [ ]:
# the model acheives 90% accuracy, save it if not saved yet
if not os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.save(MODEL_NAME)

In [ ]:
# ---------------------
# load testing data
# ---------------------
def load_test_data(load_exist):
    testset = None
    if load_exist and os.path.exists('testset-{}.npy'.format(IMG_SIZE)):
        # if want to load an existing testing set
        testset = np.load('testset-{}.npy'.format(IMG_SIZE), allow_pickle = True)
    else: 
        testset = process_test_data()
    return testset

In [ ]:
test_data = load_test_data(False)
test_x = np.array([i[0] for i in test_data]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
test_y = [i[1] for i in test_data] 

predictions = model.predict(test_x)
total_test_samples = 0 #len(test_data)

counter = 0
for prediction, true_label in zip(predictions, test_y):
    total_test_samples +=1
    predicted_class = np.argmax(prediction) # [0.6, 0.4] #predicts whichever is max
    actual_class = np.argmax(true_label)
    print("predicted:" + str(prediction))
    print("true_lable:"+str(true_label))# [0, 1], [1, 0]
    if(predicted_class == actual_class):
        counter+=1
        
    print("predicate class: {}, True label: {}, counter {} ".format(predicted_class, actual_class, counter))

# For integers, ‘/’ operator returns the quotient. For example, 3/10 returns 0.

# So, add 1.0 before counter to convert it to a float number. 

accuracy = 1.0 * counter / total_test_samples
print(total_test_samples)

print("The testing accuracy is {}.".format(accuracy))


In [ ]:
# -------------------
# use trained model to predict given image
# -------------------
import matplotlib.pyplot as plt

test_data = load_test_data(True)
fig = plt.figure(figsize = (10, 8))

# test and plot 12 images
for i, data in enumerate(test_data[:12]):
    # cat: [1, 0]; dog: [0, 1]
    img_data = data[0]
    img_idx = data[1]
    
    sub_fig = fig.add_subplot(3, 4, i + 1)
    
    orig_data = img_data
    # flatten image
    data = img_data.reshape(IMG_SIZE, IMG_SIZE, 1)
    
    output = model.predict([data])[0]
    
    if np.argmax(output) == 1:
        pred_label = 'dog'
    else:
        pred_label = 'cat'
    
    sub_fig.imshow(orig_data, cmap = 'gray')
    
    # predicted label and confidence
    plt.title(pred_label + '; ' + str(np.max(output)))
    sub_fig.get_xaxis().set_visible(False)
    sub_fig.get_yaxis().set_visible(False)
    
plt.show()